In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_ollama import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain  # ✅ Add this import


In [2]:
# import os
# from dotenv import load_dotenv

In [3]:
# load_dotenv()
# HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"] 
# HUGGINGFACEHUB_API_TOKEN

In [4]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"]= HUGGINGFACEHUB_API_TOKEN

In [5]:
# from langchain.document_loaders import PyPDFLoader

In [6]:
# %pwd

In [7]:
# %cd python


In [8]:
# %cd Generative Ai

In [9]:
# %cd BAPPY CODES

In [10]:
# %pwd

In [11]:
# import pypdf


In [12]:
# file_path = "/Users/ashishgutte/Downloads/python/Genrative AI/bappy codes/practice_codes/data/SDG.pdf"
# loader = PyPDFLoader(file_path)
# data = loader.load()

In [13]:
# data

In [14]:
# len(data)

In [15]:
# question_gen = ""
# for page in data:
#     question_gen += page.page_content

In [16]:
# question_gen

In [17]:
# from langchain.text_splitter import TokenTextSplitter

In [18]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter_ques_gen = RecursiveCharacterTextSplitter(
#     chunk_size=10000,
#     chunk_overlap=200
# )


In [19]:
# document_answer_gen = splitter_ans_gen.split_documents(
#     document_ques_gen
# )

In [20]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama  # Use Ollama instead of OpenAI


In [21]:
# Load PDF
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [22]:
# Extract text from PDF pages
question_gen = ""
for page in data:
    question_gen += page.page_content

In [23]:
# Split text into chunks
splitter_ques_gen = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=200
)
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [24]:
# Convert chunks into LangChain document format
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]
document_ques_gen


[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [25]:
# # Initialize Ollama model
# llm_ques_gen_pipeline = ChatOllama(
#     model="ollama3.2",  
#     temperature=0.3
# )

In [26]:
from langchain_community.llms import Ollama

# Initialize Ollama with your chosen model
llm_ques_gen_pipeline = Ollama(model="llama3.2")

/var/folders/m7/p03s6y6x1f56_5gbhgn0yr9h0000gn/T/ipykernel_1427/1564218756.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_ques_gen_pipeline = Ollama(model="llama3.2")


In [27]:

# Prompt Templates
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [28]:
from langchain.prompts import PromptTemplate

PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = """
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones (only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
"""
REFINE_PROMPT_QUESTIONS = PromptTemplate(input_variables=["existing_answer", "text"], template=refine_template)


In [29]:
from langchain.chains.summarize import load_summarize_chain

# Question Generation Chain
ques_gen_chain = load_summarize_chain(
    llm=llm_ques_gen_pipeline, 
    chain_type="refine", 
    verbose=True, 
    question_prompt=PROMPT_QUESTIONS, 
    refine_prompt=REFINE_PROMPT_QUESTIONS
)
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

/var/folders/m7/p03s6y6x1f56_5gbhgn0yr9h0000gn/T/ipykernel_1427/1341798389.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough 
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew 
that earthquakes and floods were inevitable, but that the high death 
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from

In [30]:
pip install sentence-transformers


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:



# Use Hugging Face Embeddings instead of OpenAI
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS Vector Store (ensure document_answer_gen is defined properly)
document_answer_gen = [Document(page_content=t) for t in chunk_ques_gen]  # Assuming it's the same as question docs
vector_store = FAISS.from_documents(document_answer_gen, embeddings)






/var/folders/m7/p03s6y6x1f56_5gbhgn0yr9h0000gn/T/ipykernel_1427/2967718004.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/ashishgutte/Downloads/python/Genrative AI/bappy codes/practice_codes/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
# Initialize Ollama for Answer Generation
# llm_answer_gen = ChatOllama(model="llama3.2", temperature=0.1)  # Replacing ChatOpenAI
llm_answer_gen = Ollama(model="llama3.2")
# Convert questions into a list
ques_list = ques.split("\n")
ques_list

['Based on the provided text and refined context, here are the revised practice questions to help prepare coders/programmers for their tests:',
 '',
 '**Section 1: Understanding Sustainable Development Goals**',
 '',
 "1. What is the main objective of the United Nations' Sustainable Development Goals (SDGs)?",
 '2. How many SDGs are there in total, and what are they?',
 '3. What does the phrase "leaving no one behind" mean in the context of SDGs?',
 '',
 '**Section 2: Access to Education**',
 '',
 '1. According to the text, how has access to electricity increased over the past few decades? Provide a specific statistic or trend.',
 '2. How do forests and plant life contribute to food production, and what percentage of the human diet does they make up?',
 '3. What is the current rate of desertification, and how can it be addressed according to the SDGs?',
 '',
 '**Section 3: Terrestrial Ecosystems**',
 '',
 "1. What percentage of the Earth's surface are forests, which cover around what a

In [33]:
# Retrieval QA Chain
answer_generation_chain = RetrievalQA.from_chain_type(
    llm=llm_answer_gen, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever()
)


In [34]:
# Answer each question and save to a file
with open("answers.txt", "w") as f:  # Open file in write mode
    for question in ques_list:
        print("Question:", question)
        answer = answer_generation_chain.run(question)
        print("Answer:", answer)
        print("--------------------------------------------------\n")

        # Save answer to file
        f.write("Question: " + question + "\n")
        f.write("Answer: " + answer + "\n")
        f.write("--------------------------------------------------\n\n")

Question: Based on the provided text and refined context, here are the revised practice questions to help prepare coders/programmers for their tests:
Answer: Here are some practice questions based on the provided text:

**Multiple Choice Questions**

1. According to the Sustainable Development Goals (SDGs), what percentage of fish stocks are overexploited?
a) 10%
b) 25%
c) 33%
d) 50%

Answer: c) 33%

2. What is the target for managing and protecting life below water, as per SDGs?
a) To reduce marine pollution by 20% by 2030
b) To increase marine conservation efforts by 30% by 2030
c) To restore and promote sustainable use of terrestrial ecosystems by 2030
d) To combat desertification by reducing greenhouse gas emissions

Answer: b) To increase marine conservation efforts by 30% by 2030

3. What is the goal for governments and communities to find lasting solutions to conflict and insecurity?
a) To strengthen institutions of global governance
b) To reduce the flow of illicit arms
c) To p